In [1]:
# Import modules
import os
import json

import pandas as pd

In [2]:
os.getcwd()

'C:\\Users\\Me\\Documents\\cc-shopee-code-league-2021\\notebooks'

In [4]:
raw_data = pd.read_json("../data/raw/contacts.json")

In [5]:
raw_data.shape

(500000, 5)

In [6]:
raw_data.head(10)

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,
5,5,,840113148,0,
6,6,hESiBgYTxMnj@hotmail.com,,4,
7,7,hqyunTiaXfIjhO@yahoo.com,,1,vxjYZtMUvNhtEBDruzQfjpsmX
8,8,XGUjcSiDpFKWEUCs@qq.com,223604620644,2,
9,9,osIIQgAiZX@hotmail.com,,0,IqewrMXVTJqSsVklWFfOpdRNI


In [13]:
df = raw_data.copy()

In [14]:
# Calculate unique counts for 3 channels (Email/Phone/OrderId)
email_count = df["Email"].value_counts().rename_axis('Email').reset_index(name='email_counts')
phone_count = df["Phone"].value_counts().rename_axis('Phone').reset_index(name='phone_counts')
order_count = df["OrderId"].value_counts().rename_axis('OrderId').reset_index(name='order_counts')

In [15]:
# Flag `is_{channel}_unique` as True if it == "" or its count == 1
email_count["is_email_unique"] = (email_count["Email"].str.len() == 0) | (email_count["email_counts"] == 1)
phone_count["is_phone_unique"] = (phone_count["Phone"].str.len() == 0) | (phone_count["phone_counts"] == 1)
order_count["is_order_unique"] = (order_count["OrderId"].str.len() == 0) | (order_count["order_counts"] == 1)

In [16]:
# Merge unique flags into df
df = df.merge(email_count, how='left', on='Email')
df = df.merge(phone_count, how='left', on='Phone')
df = df.merge(order_count, how='left', on='OrderId')

In [17]:
# Flag `is_unique` as True if flags for 3 channels are all True
df["single_channel"] = df["is_email_unique"] & df["is_phone_unique"] & df["is_order_unique"]

In [18]:
df

,Id,Email,Phone,Contacts,OrderId,email_counts,is_email_unique,phone_counts,is_phone_unique,order_counts,is_order_unique,single_channel
0,0,gkzAbIy@qq.com,,1,,1,True,254919,True,256456,True,True
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,151394,True,1,True,3,False,False
2,2,,9125983679,0,,151394,True,2,False,256456,True,False
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,1,True,254919,True,1,True,True
4,4,,300364407,2,,151394,True,1,True,256456,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499995,,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv,151394,True,1,True,1,True,True
499996,499996,,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS,151394,True,1,True,1,True,True
499997,499997,,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO,151394,True,1,True,1,True,True
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku,1,True,1,True,2,False,False


In [19]:
single_channel_contacts = df[raw_data.columns][df["single_channel"] == True]
multi_channel_contacts = df[raw_data.columns][df["single_channel"] == False]

In [20]:
single_channel_contacts

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,
7,7,hqyunTiaXfIjhO@yahoo.com,,1,vxjYZtMUvNhtEBDruzQfjpsmX
16,16,,8888238873,0,
...,...,...,...,...,...
499993,499993,qXUQsYAv@yahoo.com,,4,lByyLeHAiEctrjVSNHuMZBLgO
499995,499995,,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO


In [21]:
multi_channel_contacts

,Id,Email,Phone,Contacts,OrderId
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
5,5,,840113148,0,
6,6,hESiBgYTxMnj@hotmail.com,,4,
8,8,XGUjcSiDpFKWEUCs@qq.com,223604620644,2,
...,...,...,...,...,...
499988,499988,qJXiiwTMjRjIYe@hotmail.com,,1,AveIKdquxHFgCrNqPaihhCNGf
499989,499989,BvqHOhvgL@qq.com,,3,FAnuKqMMcznzQcHcDjprAgdOv
499990,499990,,880752090156,2,ElrIJOTvTabhzXnSMXOLtUibu
499994,499994,btjvtrrKGFKcqgop@qq.com,,0,FzJqceEASdmnduzOEHvqxdOpR


In [25]:
single_channel_contacts.to_csv("../data/interim/single_channel_contacts.csv")
multi_channel_contacts.to_csv("../data/interim/multi_channel_contacts.csv")